In [121]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import shapiro
import math
from operator import itemgetter
from pathlib import Path
import json

import warnings
warnings.filterwarnings('ignore')


# Step 1: Read Files

In [122]:
def readFiles(path_x, path_y):
    core_dataset1 = pd.read_csv(path_x, error_bad_lines=False, sep=';')
    core_dataset2 = pd.read_csv(path_y, error_bad_lines=False, sep=';')
    # core_dataset3 = pd.read_csv("C:/Users/Elitsa/Desktop/Thesis/RealData/TRD/core_2019-01.csv", error_bad_lines=False, sep=';')

    df1 = pd.DataFrame(core_dataset1)
    df2 = pd.DataFrame(core_dataset2)

    return [df1,df2]


In [123]:
read_months_of_interest = readFiles("C:/Users/Elitsa/Desktop/Thesis/RealData/TRD/core_2018-11.csv","C:/Users/Elitsa/Desktop/Thesis/RealData/TRD/core_2018-12.csv")
read_month_x = read_months_of_interest[0]
read_month_y = read_months_of_interest[1]


# Step 2: Filter Data

In [124]:
def filterData(month_x,month_y,unnecessary_columns):
    clean_month_x = month_x.drop(unnecessary_columns, axis=1)
    clean_month_y = month_y.drop(unnecessary_columns, axis=1)  
    
    return [clean_month_x, clean_month_y]
    

In [125]:
unnecessary_columns = ['job_id', 'position_id', 'first_name', 'last_name', 'gender','position_grade', 'date_of_birth', 'date_in_position', 'date_in_service', 'dotted_line', 'functional_area_level0', 'functional_area_level1', 'functional_area_level2', 'functional_area_level3', 'functional_area_level4', 'functional_area_level5', 'location_level0', 'location_level1', 'location_level2', 'location_level3', 'location_level4', 'location_level5', 'legal_entity_level0', 'legal_entity_level1', 'legal_entity_level2', 'legal_entity_level3', 'legal_entity_level4', 'cost_center_level0', 'cost_center_level1', 'cost_center_level2', 'cost_center_level3', 'cost_center_level4', 'cost_center_level5', 'cost_center_level6', 'cost_center_level7', 'contract_type', 'mobility', 'retention_risk', 'retention_risk_reason', 'solid_line_layer', 'hire_type', 'leave_type', 'target_bonus', 'custom_intern_extern', 'custom_type_management', 'custom_zvp_mdw', 'custom_tpow', 'added_as_hire', 'removed_as_leaver', 'hire_reason_level0', 'leave_reason_level0', 'regrettable', 'employee_status', 'hire_channel']

clean_months = filterData(read_month_x,read_month_y,unnecessary_columns)
clean_month_x = clean_months[0]
clean_month_y = clean_months[1]

# Step 3: Extract Changes

## Helpers 

In [126]:
def report_diff(x):
    if x[0] == x[1]:
        return x[0]
    elif not(isinstance(x[0], float) and math.isnan(x[0])) and not(isinstance(x[1], float) and math.isnan(x[1])):
        return "Changed"

def getDatasetChanges(dataset):
    dataset_changes = {}
    for col, temp in dataset.iterrows():
        temp_1 = temp[(temp == "Changed")]
        if(len(temp_1) > 1):
            dataset_changes[(temp_1).name] = list(temp_1.to_dict().keys())
    return dataset_changes

def getChangingValues(employee_id,columns,dataset):
    field_with_values = {}
    for col in columns:  
        field_with_values[col] = dataset.loc[dataset['employee_id'] == employee_id, col].iloc[0]      
    return field_with_values

def getChangesInDataset(dataset_changes,original_dataset):
    dataset_changes_values = {} 
    for employee_id, columns in dataset_changes.items(): 
        dataset_changes_values[employee_id] = getChangingValues(employee_id,columns,original_dataset)
        
    df = pd.DataFrame.from_dict(dataset_changes_values,orient="index")
    return df


## Main function

In [159]:
def extractChanges(month_x,month_y,unique_identifier):
    
    month_x['version'] = "old"
    month_y['version'] = "new"
    
    full_set = pd.concat([month_x,month_y],ignore_index=True)
    needed_columns = list(month_y.columns.values)

    changes = full_set.drop_duplicates(subset=needed_columns, keep='last' )

    dupe_accts = changes.set_index(unique_identifier).index.get_duplicates()
    dupes = changes[changes[unique_identifier].isin(dupe_accts)]

    change_month_y = dupes[(dupes["version"] == "new")]
    change_month_x = dupes[(dupes["version"] == "old")]

    change_month_y = change_month_y.drop(['version'], axis=1)
    change_month_x = change_month_x.drop(['version'], axis=1)

    change_month_y.set_index(unique_identifier,inplace=True)
    change_month_x.set_index(unique_identifier,inplace=True)
    
    diff_panel = pd.Panel(dict(df1=change_month_y,df2=change_month_x))
    diff_output = diff_panel.apply(report_diff, axis=0)
    
    dataset_changes_only = getDatasetChanges(diff_output)
        
    return diff_output

In [163]:
diff_output = extractChanges(clean_month_x, clean_month_y, 'employee_id')

dataset_changes_only = getDatasetChanges(diff_output)

dataset_values_for_two_months = [getChangesInDataset(dataset_changes_only,month_x),getChangesInDataset(dataset_changes_only,month_y)]

diff_output

,fte,employee_grade,title,solid_line,business_unit_level0,business_unit_level1,business_unit_level2,business_unit_level3,business_unit_level4,business_unit_level5,talent_status,potential,performance_status,base_salary,currency,relative_salary_position
employee_id,,,,,,,,,,,,,,,,
9,0.8421,7,Coordinator B Ruimtebeheer,11075,COO Zakelijk,Services,Facility & Support,Huisvesting,None,None,8. Goede presteerder,Laag,Medium,50382.3,EUR,1
24,1,12,Teammanager E,20862,COO Zakelijk,AOV,"AOV Individueel, Med.&Arbeidsdesk.Dienst",AOV Operations Individueel,None,None,9. Juiste persoon op de juiste plek,Laag,Hoog,99487.9,EUR,1.02001
47,0.8421,4,Support Medewerker B Administratie,20109,COO Zakelijk,AOV,"AOV Individueel, Med.&Arbeidsdesk.Dienst",AOV Operations Individueel,Back-Office Service,None,9. Juiste persoon op de juiste plek,Laag,Hoog,36809.4,EUR,0.999998
49,0.5263,9,Analist D Business Risk,19085,CFO,Finance & Risk,Finance & Risk AOV,Business Control & Riskmanagement AOV,None,None,8. Goede presteerder,Laag,Medium,63307.9,EUR,1
72,0.7,6,Klantbeheerder C Incasso,279962,COO Zakelijk,Ziektekosten,Operatie en Informatiemanagement Zorg,Debiteurenbeheer Zorg,None,None,6. Professioneel Talent,Medium,Hoog,48210.8,EUR,1.0696
95,1,10,Teammanager C,5086,COO Zakelijk,Ziektekosten,Operatie en Informatiemanagement Zorg,Acceptatie Ziektekosten Team 1,None,None,8. Goede presteerder,Laag,Medium,64874.9,EUR,0.902461
122,1,8,Specialist B Project Expertise,11351,COO Particulier,Pensioenen,Bestaande Portefeuille Pensioenen,Projectexpertise DAV,None,None,8. Goede presteerder,Laag,Medium,56005.5,EUR,1
129,1,8,Ontwerper & Ontwikkelaar B Modelleur,585149,COO Zakelijk,AOV,Informatiemanagement AOV,SK AOV Change,None,None,8. Goede presteerder,Laag,Medium,56643.7,EUR,1.0114
149,1,23,Directeur,18862,COO Zakelijk,AOV,None,None,None,None,None,None,None,None,EUR,None


# Step 4: Outlier Detection

## Helpers

In [ ]:
all_columns = ['employee_grade',
       'relative_salary_position', 'business_unit_level2',
       'business_unit_level3', 'base_salary', 'fte', 'title',
       'business_unit_level0', 'business_unit_level1', 'solid_line',
       'business_unit_level4']

categorical = ['employee_grade', 'business_unit_level2',
       'business_unit_level3','title',
       'business_unit_level0', 'business_unit_level1', 'solid_line',
       'business_unit_level4']

numeric = ['relative_salary_position','fte','base_salary']

ordinal = ['employee_grade']

# Numerical Data
def checkNormality(points, alpha = 0.05):
    stat, p = shapiro(points)
    if p > alpha:
        return True 
    else:
        return False
    
def MADbasedOutlierDetection(points, thresh=3.5):
    normality = checkNormality(points) 
    outliers = []
    median = np.median(points, axis=0)
    absolute_deviations = np.absolute(points-median)
    median_abs_deviation = np.median(absolute_deviations)
    
    if(normality is False):
        sd = np.std(points)
        b = sd/median_abs_deviation
    else:
        b = 1.4826
        
    mad = b * median_abs_deviation
    
    lower_limit = median - (thresh*mad)
    upper_limit = median + (thresh*mad)
    for point in points:
        if point < lower_limit or point > upper_limit:
            outliers.append(point)
            
    return outliers

# Categorical Data
# Returns the frequencies of categories that should be considered outliers
def categoricalOutlierDetection(frequencies):
    outlier_frequencies = list(frequencies.to_dict().values())
    mean = np.mean(outlier_frequencies)
    half_mean = round(mean/2)
    categorical_outliers = []
    for frequency in outlier_frequencies:
        if(frequency < half_mean):
            categorical_outliers.append(frequency)     
    return list(set(categorical_outliers))

# Returns the outlier categories
def getOutlierCategories(outlier_frequencies,frequencies): # 1st - list, 2 - dict
    categories = []
    for key, value in frequencies.items():
        for outlier in outlier_frequencies:
            if(value == outlier):
                categories.append(key)
    return categories

def detectOutliersInChanges(outliers,changes):
    returned_outliers = []
    for change in changes:
        if(change in outliers):
            returned_outliers.append(change)
    return returned_outliers

def checkPreviousMonth(employee_id,field,dataset):
    
    checked_outlier_value = dataset.loc[dataset['employee_id'] == employee_id, field].iloc[0]
   
    if(field in categorical):
        value_counts = dataset[field].value_counts()
        fd_outliers = categoricalOutlierDetection(value_counts)
        outlier_cats = getOutlierCategories(fd_outliers,value_counts)
        if(checked_outlier_value in outlier_cats):
            return True
    else:
        values_in_full_dataset = dataset[field].values
        values_in_full_dataset = values_in_full_dataset[np.logical_not(np.isnan(values_in_full_dataset))]
        value_outliers = MADbasedOutlierDetection(values_in_full_dataset)
        if(checked_outlier_value in value_outliers):
            return True
    return False


## Main function

In [167]:
def outlierAndChangesDetection(dataset_changes,original_dataset,old_dataset):
    all_outliers = {}
    for i, j in dataset_changes.iterrows():
        changes_dict = j.to_dict()
        outliers_and_changes = {}
        employee_id = i
        outliers_arr = []
        changes_arr = []
        old_outliers = []
        for key, val in changes_dict.items():
            last_month_outlier = checkPreviousMonth(employee_id,key,old_dataset)
            if(key != 'employee_id'):
                if(key in numeric):
                    if(not(math.isnan(float(val)))):
                        values_in_full_dataset = original_dataset[key].values
                        values_in_full_dataset = values_in_full_dataset[np.logical_not(np.isnan(values_in_full_dataset))]
                        value_outliers = MADbasedOutlierDetection(values_in_full_dataset)
                        if(val in value_outliers):
                            if(last_month_outlier == False):
                                outliers_arr.append(key)
                                outliers_and_changes["outliers"] = outliers_arr
                            else:
                                old_outliers.append(key)
                                outliers_and_changes["old_outliers"] = old_outliers
                        else:
                            if(isinstance(val, float) and not(math.isnan(val))):
                                changes_arr.append(key)
                            elif(not(isinstance(val,float))):
                                changes_arr.append(key)
                            outliers_and_changes["changes"] = changes_arr
                else:
                    value_counts = original_dataset[key].value_counts()
                    fd_outliers = categoricalOutlierDetection(value_counts)
                    outlier_cats = getOutlierCategories(fd_outliers,value_counts)
                    changes_in_values = dataset_changes[key].values
                    result = detectOutliersInChanges(outlier_cats,changes_in_values)
                    if(val in result):
                        if(last_month_outlier == False):
                            outliers_arr.append(key)
                            outliers_and_changes["outliers"] = outliers_arr
                        else:
                            changes_arr.append(key)
                            old_outliers.append(key)
                            outliers_and_changes["old_outliers"] = old_outliers
                    else:
                        if(isinstance(val, float) and not(math.isnan(val))):
                            changes_arr.append(key)
                        elif(not(isinstance(val,float))):
                            changes_arr.append(key)
                        outliers_and_changes["changes"] = changes_arr
            if(len(outliers_and_changes) > 0):
                all_outliers[employee_id] = outliers_and_changes
            
    return all_outliers


In [168]:
outliers_and_changes = outlierAndChangesDetection(dataset_values_for_two_months[0],clean_month_x, clean_month_y)
outliers_and_changes

{1170: {'changes': ['relative_salary_position']},
 6868: {'changes': ['relative_salary_position', 'base_salary'],
  'outliers': ['business_unit_level2', 'business_unit_level3']},
 10709: {'changes': ['relative_salary_position', 'base_salary', 'fte']},
 12077: {'changes': ['relative_salary_position', 'fte']},
 15678: {'changes': ['relative_salary_position', 'base_salary', 'title']},
 16325: {'changes': ['fte', 'business_unit_level0'],
  'outliers': ['business_unit_level1']},
 19656: {'changes': ['relative_salary_position',
   'business_unit_level3',
   'base_salary',
   'title',
   'business_unit_level0',
   'business_unit_level1'],
  'outliers': ['business_unit_level2']},
 19743: {'changes': ['business_unit_level4'], 'outliers': ['solid_line']},
 20002: {'changes': ['business_unit_level2', 'fte', 'title'],
  'outliers': ['business_unit_level3'],
  'old_outliers': ['title']},
 20687: {'changes': ['relative_salary_position', 'base_salary', 'title']},
 21050: {'changes': ['business_unit_l

# Step 5: Outlier Explanation

## Helpers

In [169]:
# Two categorical values 
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))


# 1 categorical, 1 numeric
def correlation_ratio(categories, measurements):
    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat)+1
    y_avg_array = np.zeros(cat_num)
    n_array = np.zeros(cat_num)
    
    for i in range(0,cat_num):
        cat_measures = measurements[np.argwhere(fcat == i).flatten()]
        n_array[i] = len(cat_measures)
        y_avg_array[i] = np.average(cat_measures)
        
    y_total_avg = np.nansum(np.multiply(y_avg_array,n_array))/np.nansum(n_array)
    numerator = np.nansum(np.multiply(n_array,np.power(np.subtract(y_avg_array,y_total_avg),2)))
    denominator = np.nansum(np.power(np.subtract(measurements,y_total_avg),2))
    if numerator == 0:
        eta = 0.0
    else:

        eta = np.sqrt(numerator/denominator)
    return eta

def calculateCorrelation(specific_column,columns,dataset):
    correlations = {}
    specific_column_numeric = False
    col_type_numeric = False
    if(specific_column in numeric):
        specific_column_numeric = True
    else:
        specific_column_numeric = False
    for col in columns:
        correlation = 0
        if(col in numeric):
            col_type_numeric = True
        elif(col in categorical):
            col_type_numeric = False
         
        if(not(col == specific_column)):
            if(specific_column_numeric == True and col_type_numeric == True):
                df = dataset[[specific_column,col]]
                df_no_nans = df.dropna()
                correlation = round(abs(stats.pearsonr(df_no_nans[specific_column],df_no_nans[col])[0]),3)
            elif(specific_column_numeric == True and col_type_numeric == False):
                correlation = round(correlation_ratio(dataset[col],dataset[specific_column]),3)
            elif(specific_column_numeric == False and col_type_numeric == True):
                correlation = round(correlation_ratio(dataset[specific_column],dataset[col]),3)
            elif(specific_column_numeric == False and col_type_numeric == False):
                correlation = round(cramers_v(dataset[specific_column],dataset[col]),3)
           
            correlations[col] = correlation 
              
        
    return correlations

def calculateEveryCorrelations(columns,dataset):
    correlations = {}
    for col in columns:
        all_correlations_for_field = calculateCorrelation(col,columns,dataset)
        correlations[col] = all_correlations_for_field       
    return correlations

def possibleExplanationsCorrelation(all_correlations,outliers,changes):
    ranked_explanations = {}
    for outlier in outliers:
        reasons = []
        all_changes = np.concatenate([changes,outliers])
        sorted_correlations = sorted(all_correlations[outlier].items(),key=itemgetter(1),reverse = True) 
        for sc in sorted_correlations:
            if(sc[0] in all_changes):
                reasons.append(sc)

        ranked_explanations[outlier] = reasons
        
    return ranked_explanations      
    
def explainingOutliersForEmployees(correlations,outliers_and_changes):
    outliers = {} 
    result = {}
    
    for employee_id, changes_and_outliers in outliers_and_changes.items():
        for diff, fields in changes_and_outliers.items():
            if(diff == "outliers"):
                possible_explanations = possibleExplanationsCorrelation(correlations,fields,changes_and_outliers['changes'])
                result[employee_id] = possibleExplanationsCorrelation(correlations,fields,changes_and_outliers['changes'])      

    return result



In [170]:
all_correlations = calculateEveryCorrelations(all_columns,clean_month_x)
explanations = explainingOutliersForEmployees(all_correlations,outliers_and_changes)

# explanations


# Step 6: Extract Valuable Changes

## Helpers

In [171]:
def estimateTheAveragesNumericOfAllColumn(new_dataset):
    averages = {}
    all_columns = list(new_dataset.columns.values)
    for col in all_columns:
        if col in numeric:
            average = new_dataset[col].mean()
            averages[col] = average
    return round(averages,3)

def estimateThePercentageChange(average,number):
    if (average == 0.0 or number == 0.0):
        return 0
    change = (((number-average)/average)*100)
    return round(change,3)

def isSignificant(change,threshold = 50):
    if(change >= threshold or change <= -abs(threshold)):
        return True
    return False
    


## Main Function

In [189]:
def interestingChanges(ids_and_changes,new,old,dataset_changes):
    interesting_changes_for_employee = []
    for employee_id, changes_and_outliers in ids_and_changes.items():
        if(not(changes_and_outliers.get("outliers"))): # Without outliers 
            est = {}
            for change in changes_and_outliers['changes']:
                estimations = {}
                if(change in ordinal):
                    change_now = old.loc[old['employee_id'] == employee_id, change].iloc[0]
                    change_before = new.loc[new['employee_id'] == employee_id, change].iloc[0]
                    change_now2 = employee_grade[change_now]
                    change_before2 = employee_grade[change_before]
                    if(change_now2 != change_before2):
                        percentage_change = estimateThePercentageChange(float(change_before2),float(change_now2))
                        if(abs(float(change_before2) - float(change_now2)) > 2 ): 
                            formatted = str(employee_grade.get(str(change_before2))) + " -> " + str(employee_grade.get(str(change_now)))
                            estimations['change'] = formatted
                            if(percentage_change != 0):
                                estimations['percentage_change'] = percentage_change
                        
                elif(change in numeric):
                        
                        change_before = round((old.loc[old['employee_id'] == employee_id, change].iloc[0]),3)
                        change_now = round((new.loc[new['employee_id'] == employee_id, change].iloc[0]),3)
                        if(change_now != change_before):
                            formatted = str(change_before) + " -> " + str(change_now)
                            average_change = round(dataset_changes[change].mean(),3)
                            compared_to_average = estimateThePercentageChange(average_change,change_now)
                            percentage_change = estimateThePercentageChange(change_before,change_now)
                            if(isSignificant(percentage_change) or isSignificant(compared_to_average)):
                                
                                estimations['change'] = formatted
                                estimations['average_change'] = average_change
                                estimations['compared_to_average'] = compared_to_average
                                estimations['percentage_change'] = percentage_change
  
                else:
                    estimations = []

                if(len(estimations) > 0):
                    est['employee_id'] = employee_id
                    est['changes'] = {change : estimations}
                  
   
            if(len(est) > 0):
               
                interesting_changes_for_employee.append(est)
                
    return interesting_changes_for_employee

month_x['employee_grade'] = month_x['employee_grade'].map({'1': 1, '2': 2, '3': 3,'4': 4, '5': 5, '6': 6,'7': 7, '8': 8, '9': 9,'10': 10, '11': 11, '12': 12, '13': 13, '14': 14,'15': 15, '22': 16, '23': 17,'AM-11': 18, 'AM-12': 19, 'AM-13': 20, 'AM-14': 21, 'AM-15': 22, 'EUR-01': 23,'BUN-06' : 24, 'BUN-09': 25, 'BUN-10': 26, 'BUN-11': 27, 'BUN-12': 28,'BUO-06': 29, 'BUO-07': 30, 'BUO-07A': 31, 'BUO-09': 32, 'BUO-10': 33, 'BUO-10A': 34,'BUO-11' : 35, 'BUO-12': 36})
month_y['employee_grade'] = month_y['employee_grade'].map({'1': 1, '2': 2, '3': 3,'4': 4, '5': 5, '6': 6,'7': 7, '8': 8, '9': 9,'10': 10, '11': 11, '12': 12, '13': 13, '14': 14,'15': 15, '22': 16, '23': 17,'AM-11': 18, 'AM-12': 19, 'AM-13': 20, 'AM-14': 21, 'AM-15': 22, 'EUR-01': 23,'BUN-06' : 24, 'BUN-09': 25, 'BUN-10': 26, 'BUN-11': 27, 'BUN-12': 28,'BUO-06': 29, 'BUO-07': 30, 'BUO-07A': 31, 'BUO-09': 32, 'BUO-10': 33, 'BUO-10A': 34,'BUO-11' : 35, 'BUO-12': 36})

# df3['employee_grade'] = df3['employee_grade'].map({'0': 1, '2': 2, '3': 3,'4': 4, '5': 5, '6': 6,'7': 7, '8': 8, '9': 9,'10': 10, '11': 11, '12': 12, '13': 13, '14': 14,'15': 15, '22': 16, '23': 17,'AM-11': 18, 'AM-12': 19, 'AM-13': 20, 'AM-14': 21, 'AM-15': 22, 'EUR-01': 23,'BUN-06' : 24, 'BUN-09': 25, 'BUN-10': 26, 'BUN-11': 27, 'BUN-12': 28,'BUO-06': 29, 'BUO-07': 30, 'BUO-07A': 31, 'BUO-09': 32, 'BUO-10': 33, 'BUO-10A': 34,'BUO-11' : 35, 'BUO-12': 36})
# df3['employee_grade'].value_counts()
employee_grade = {'0': 0, '1': 1, '2': 2, '3': 3,'4': 4, '5': 5, '6': 6,'7': 7, '8': 8, '9': 9,'10': 10, '11': 11, '12': 12, '13': 13, '14': 14,'15': 15, '22': 16, '23': 17,'AM-11': 18, 'AM-12': 19, 'AM-13': 20, 'AM-14': 21, 'AM-15': 22, 'EUR-01': 23,'BUN-06' : 24, 'BUN-09': 25, 'BUN-10': 26, 'BUN-11': 27, 'BUN-12': 28,'BUO-06': 29, 'BUO-07': 30, 'BUO-07A': 31, 'BUO-09': 32, 'BUO-10': 33, 'BUO-10A': 34,'BUO-11' : 35, 'BUO-12': 36}

In [191]:
dataset_changes = diff_output[diff_output['currency'].str.contains("Changed") | diff_output['relative_salary_position'].str.contains('Changed') | diff_output['performance_status'].str.contains("Changed") | diff_output['base_salary'].str.contains('Changed') | diff_output['talent_status'].str.contains("Changed") | diff_output['potential'].str.contains('Changed') | diff_output['fte'].str.contains("Changed") | diff_output['employee_grade'].str.contains('Changed') |  diff_output['title'].str.contains('Changed') | diff_output['solid_line'].str.contains('Changed') | diff_output['business_unit_level0'].str.contains('Changed') | diff_output['business_unit_level1'].str.contains("Changed") | diff_output['business_unit_level2'].str.contains("Changed") | diff_output['business_unit_level3'].str.contains("Changed") | diff_output['business_unit_level4'].str.contains("Changed") | diff_output['business_unit_level5'].str.contains('Changed') ] 

dataset_changes
clean_month_y
interesting_changes = interestingChanges(outliers_and_changes,clean_month_y,clean_month_x,dataset_values_for_two_months[1])
interesting_changes

[{'employee_id': 10709,
  'changes': {'fte': {'change': '0.5 -> 0.842',
    'average_change': 0.77,
    'compared_to_average': 9.351,
    'percentage_change': 68.4}}},
 {'employee_id': 279517,
  'changes': {'fte': {'change': '0.947 -> 0.2',
    'average_change': 0.77,
    'compared_to_average': -74.026,
    'percentage_change': -78.881}}},
 {'employee_id': 752363,
  'changes': {'base_salary': {'change': '86063.066 -> 88644.973',
    'average_change': 51826.526,
    'compared_to_average': 71.042,
    'percentage_change': 3.0}}}]

        employee_grade  relative_salary_position  \
1170               NaN                  0.954314   
6868               NaN                  0.970995   
10709              NaN                  1.000005   
12077              NaN                  0.999998   
15678              NaN                  0.921470   
16325              NaN                       NaN   
19656              NaN                  0.783579   
19743              NaN                       NaN   
20002              NaN                       NaN   
20687              NaN                  0.921471   
21050              NaN                       NaN   
21883              NaN                       NaN   
22132              NaN                  0.784059   
22849              NaN                  0.775927   
23509              NaN                  0.831121   
23609              NaN                  0.699997   
23895              NaN                  0.812002   
24417              NaN                  0.765876   
24564       